In [209]:
from logging import raiseExceptions
from tokenize import Double
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from vec3 import Vec3

import torch
torch.manual_seed(125)
import random
random.seed(125)
use_gpu = True
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")

In [210]:
def count_fn(f):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        return f(*args, **kwargs)
    wrapper.count = 0
    return wrapper

class Node:
    """
    Class Node
    """
    def __init__(self, value, radius, left = None, right = None, position = None, cl_prob= None, ce = None, mse = None):
        self.left = left
        self.data = value
        self.radius = radius
        self.position = position
        self.right = right
        self.prob = cl_prob
        self.mse = mse
        self.ce = ce
        self.children = [self.left, self.right]
    
    def agregarHijo(self, children):

        if self.right is None:
            self.right = children
        elif self.left is None:
            self.left = children

        else:
            raise ValueError ("solo arbol binario ")


    def is_leaf(self):
        if self.right is None:
            return True
        else:
            return False

    def is_two_child(self):
        if self.right is not None and self.left is not None:
            return True
        else:
            return False

    def is_one_child(self):
        if self.is_two_child():
            return False
        elif self.is_leaf():
            return False
        else:
            return True

    def childs(self):
        if self.is_leaf():
            return 0
        if self.is_one_child():
            return 1
        else:
            return 2
    
    
    def traverseInorder(self, root):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorder(root.left)
            print (root.data, root.radius)
            self.traverseInorder(root.right)

    def traverseInorderLoss(self, root, loss):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorderLoss(root.left, loss)
            loss.append(root.prob)
            self.traverseInorderLoss(root.right, loss)
            return loss

    def traverseInorderMSE(self, root, loss):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorderMSE(root.left, loss)
            loss.append(root.mse)
            self.traverseInorderMSE(root.right, loss)
            return loss

    def traverseInorderCE(self, root, loss):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorderCE(root.left, loss)
            loss.append(root.ce)
            self.traverseInorderCE(root.right, loss)
            return loss

    def traverseInorderChilds(self, root, l):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorderChilds(root.left, l)
            l.append(root.childs())
            self.traverseInorderChilds(root.right, l)
            return l

    def preorder(self, root):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            print (root.data, root.radius)
            self.preorder(root.left)
            self.preorder(root.right)

    def cloneBinaryTree(self, root):
     
        # base case
        if root is None:
            return None
    
        # create a new node with the same data as the root node
        root_copy = Node(root.data, root.radius)
    
        # clone the left and right subtree
        root_copy.left = self.cloneBinaryTree(root.left)
        root_copy.right = self.cloneBinaryTree(root.right)
    
        # return cloned root node
        return root_copy

    def height(self, root):
    # Check if the binary tree is empty
        if root is None:
            return 0 
        # Recursively call height of each node
        leftAns = self.height(root.left)
        rightAns = self.height(root.right)
    
        # Return max(leftHeight, rightHeight) at each iteration
        return max(leftAns, rightAns) + 1

    # Print nodes at a current level
    def printCurrentLevel(self, root, level):
        if root is None:
            return
        if level == 1:
            print(root.data, end=" ")
        elif level > 1:
            self.printCurrentLevel(root.left, level-1)
            self.printCurrentLevel(root.right, level-1)

    def printLevelOrder(self, root):
        h = self.height(root)
        for i in range(1, h+1):
            self.printCurrentLevel(root, i)


    
    def count_nodes(self, root, counter):
        if   root is not None:
            self.count_nodes(root.left, counter)
            counter.append(root.data)
            self.count_nodes(root.right, counter)
            return counter

    
    def serialize(self, root):
        def post_order(root):
            if root:
                post_order(root.left)
                post_order(root.right)
                ret[0] += str(root.data)+'_'+ str(root.radius) +';'
                
            else:
                ret[0] += '#;'           

        ret = ['']
        post_order(root)
        return ret[0][:-1]  # remove last ,

    def search(self, node, data):
        """
        Search function will search a node into tree.
        """
        # if root is None or root is the search data.
        if node is None or node.data == data:
            return node
        if node.data < data:
            return self.search(node.right, data)
        else:
            return self.search(node.left, data)

    def toGraph( self, graph, index, dec, proc=True):
        
        radius = self.radius.cpu().detach().numpy()
        if dec:
            radius= radius[0]
        print("radius", radius)
        graph.add_nodes_from( [ (index, {'posicion': radius[0:3], 'radio': radius[3] } ) ])

        if self.right is not None:
            leftIndex = self.right.toGraph( graph, index + 1, dec)

            graph.add_edge( index, index + 1 )
            if proc:
                nx.set_edge_attributes( graph, {(index, index+1) : {'procesada':False}})
        
            if self.left is not None:
                retIndex = self.left.toGraph( graph, leftIndex, dec )

                graph.add_edge( index, leftIndex)
                if proc:
                    nx.set_edge_attributes( graph, {(index, leftIndex) : {'procesada':False}})
            
            else:
               return leftIndex

        else:
            return index + 1


In [211]:
def traverse(root, tree):
       
        if root is not None:
            traverse(root.left, tree)
            tree.append((root.radius, root.data))
            traverse(root.right, tree)
            return tree

def traverse_2(tree1, tree2, t_l):
       
        if tree1 is not None:
            traverse_2(tree1.left, tree2.left, t_l)
            if tree2:
                t_l.append((tree1.radius, tree2.radius))
                print((tree1.radius, tree2.radius))
            else:
                t_l.append(tree1.radius)
                print((tree1.radius))
            traverse_2(tree1.right, tree2, t_l)
            return t_l
            

def traverse_conexiones(root, tree):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            traverse_conexiones(root.left, tree)
            if root.right is not None:
                tree.append((root.data, root.right.data))
            if root.left is not None:
                tree.append((root.data, root.left.data))
            traverse_conexiones(root.right, tree)
            return tree



In [212]:
@count_fn
def createNode(data, radius, position = None, left = None, right = None, cl_prob = None, ce = None, mse=None):
        """
        Utility function to create a node.
        """
        return Node(data, radius, position, left, right, cl_prob, ce, mse)
 
def deserialize(data):
    if  not data:
        return 
    nodes = data.split(';')  
    #print("node",nodes[3])
    def post_order(nodes):
                
        if nodes[-1] == '#':
            nodes.pop()
            return None
        node = nodes.pop().split('_')
        data = int(node[0])
        #radius = float(node[1])
        #print("node", node)
        #breakpoint()
        radius = node[1]
        #print("radius", radius)
        rad = radius.split(",")
        rad [0] = rad[0].replace('[','')
        rad [3] = rad[3].replace(']','')
        r = []
        for value in rad:
            r.append(float(value))
        #r =[float(num) for num in radius if num.isdigit()]
        r = torch.tensor(r, device=device)
        #breakpoint()
        root = createNode(data, r)
        root.right = post_order(nodes)
        root.left = post_order(nodes)
        
        return root    
    return post_order(nodes)    


def read_tree(filename):
    with open(filename, "r") as f:
        byte = f.read() 
        return byte

ENCODER

In [213]:
###ENCODER
class LeafEncoder(nn.Module):
    
    def __init__(self):
        super(LeafEncoder, self).__init__()
        self.l1 = nn.Linear(4, 32)
        self.l2 = nn.Linear(32, 32)
        self.tanh = nn.Tanh()

    def forward(self, input):
        rad = torch.tensor(input.radius)
        rad = torch.reshape(rad, (1,4)).to(device)
        radius = self.l1(rad)
        radius = self.tanh(radius)
        radius = self.l2(radius)
        radius = self.tanh(radius)
        
        return radius

class NonLeafEncoder(nn.Module):
    
    def __init__(self):
        super(NonLeafEncoder, self).__init__()
        self.l1 = nn.Linear(4,16)
        self.l2 = nn.Linear(16,32)

        self.left = nn.Linear(32,32)
        self.right = nn.Linear(32,32)
        
        self.encoder = nn.Linear(64, 32)
        self.tanh = nn.Tanh()


    def forward(self, input, left_input, right_input):
        
        radius = self.l1(torch.tensor(input.radius).reshape(1,4).to(device))
        radius = self.tanh(radius)
        radius = self.l2(radius)
        radius = self.tanh(radius)
        context = self.right(right_input)
        if left_input is not None:
            context += self.left(left_input)
        context = self.tanh(context)
    
        feature = torch.cat((radius,context), 1)
        feature = self.encoder(feature)
        feature = self.tanh(feature)


        return feature

leafenc = LeafEncoder()
nonleafenc = NonLeafEncoder()
leafenc = leafenc.to(device)
nonleafenc = nonleafenc.to(device)

In [214]:
def encode_structure_fold(root):
    
    def encode_node(node):
        
        if node is None:
            return
        if node.is_leaf():
            return leafenc(node)
        else:
            left = encode_node(node.left)
            right = encode_node(node.right)
            return nonleafenc(node, left, right)
        
    encoding = encode_node(root)
    return encoding

In [215]:
class NodeClassifier(nn.Module):
    
    def __init__(self):
        super(NodeClassifier, self).__init__()
        self.mlp1 = nn.Linear(32, 16)
        self.tanh = nn.LeakyReLU()
        self.mlp2 = nn.Linear(16, 8)

        self.tanh2 = nn.LeakyReLU()
        self.mlp3 = nn.Linear(8, 3)

    def forward(self, input_feature):
        output = self.mlp1(input_feature)
        output = self.tanh(output)
        output = self.mlp2(output)

        output = self.tanh2(output)
        output = self.mlp3(output)
        
        return output



In [216]:
class InternalDecoder(nn.Module):

    """ Decode an input (parent) feature into a left-child and a right-child feature """
    def __init__(self):
        super(InternalDecoder, self).__init__()
        self.mlp = nn.Linear(32, 32)
        self.lp2 = nn.Linear(32, 32)
        self.mlp_right = nn.Linear(32, 32)
        self.tanh = nn.Tanh()
        self.mlp2 = nn.Linear(32,4)

    def forward(self, parent_feature):
        
        vector = self.mlp(parent_feature)
        vector = self.tanh(vector)
        vector = self.lp2(vector)
        vector = self.tanh(vector)
        right_feature = self.mlp_right(vector)
        right_feature = self.tanh(right_feature)
        rad_feature = self.mlp2(vector)

        return right_feature, rad_feature

class BifurcationDecoder(nn.Module):
    
    """ Decode an input (parent) feature into a left-child and a right-child feature """
    def __init__(self):
        super(BifurcationDecoder, self).__init__()
        self.mlp = nn.Linear(32,32)
        self.lp2 = nn.Linear(32,32)
        self.mlp_left = nn.Linear(32,32)
        self.mlp_right = nn.Linear(32,32)
        self.mlp2 = nn.Linear(32,4)
        self.tanh = nn.Tanh()

    def forward(self, parent_feature):
        vector = self.mlp(parent_feature)
        vector = self.tanh(vector)
        vector = self.lp2(vector)
        vector = self.tanh(vector)
        left_feature = self.mlp_left(vector)
        left_feature = self.tanh(left_feature)
        right_feature = self.mlp_right(vector)
        right_feature = self.tanh(right_feature)
        rad_feature = self.mlp2(vector)

        return left_feature, right_feature, rad_feature



class featureDecoder(nn.Module):
    
    """ Decode an input (parent) feature into a left-child and a right-child feature """
    def __init__(self):
        super(featureDecoder, self).__init__()
        self.mlp = nn.Linear(32,32)
        self.mlp2 = nn.Linear(32,32)
        self.tanh = nn.Tanh()
        self.mlp3 = nn.Linear(32,4)

    def forward(self, parent_feature):
        vector = self.mlp(parent_feature)
        vector = self.tanh(vector)
        vector = self.mlp2(vector)
        vector = self.tanh(vector)
        vector = self.mlp3(vector)
       
        return vector

featuredec = featureDecoder()
featuredec=featuredec.to(device)
bifdec = BifurcationDecoder()
bifdec = bifdec.to(device)
internaldec = InternalDecoder()
internaldec=internaldec.to(device)
nodeClassifier = NodeClassifier()
nodeClassifier = nodeClassifier.to(device)


In [217]:
def calcularLossEstructura(cl_p, original):
    mult = torch.tensor([1/3.,1/56,1/2.], device = device)#1-7
    #mult = torch.tensor([1/3.,1/16,1/2.], device = device)#1-2
    ce = nn.CrossEntropyLoss(weight=mult)

    
    if original.childs() == 0:
        vector = [1, 0, 0] 
    if original.childs() == 1:
        vector = [0, 1, 0]
    if original.childs() == 2:
        vector = [0, 0, 1] 

    #mult = torch.tensor([1/3.,1/58,1/2.], device = device)#1-0
    #mult = torch.tensor([1/4.,1/72,1/3.], device = device)#19-2
    #mult = torch.tensor([1/2.,1/60,1/1.], device = device)#b1-0

    c = ce(cl_p, torch.tensor(vector, device=device, dtype = torch.float).reshape(1, 3))
    return c


def calcularLossAtributo(nodo, radio):
    
    radio = radio.reshape(4)
    l2    = nn.MSELoss(reduction = 'mean')
   
    mse = l2(radio, nodo.radius)
    return mse

In [218]:
def decode_structure_fold(v, root, max_nodes = 200, max_depth = 100):

    def decode_node(v, node, max_nodes, max_depth, level = 0):
        cl = nodeClassifier(v)
        _, label = torch.max(cl, 1)
        label = label.data

        
        if node.childs() == 0 and createNode.count <= max_nodes: ##output del classifier
            lossEstructura = calcularLossEstructura(cl, node)
            radio = featuredec(v)
            lossAtrs = calcularLossAtributo( node, radio )
            nd = createNode(1,radio, ce = lossEstructura,  mse = lossAtrs)
            return nd

        elif node.childs() == 1 and createNode.count <= max_nodes:
            right, radius = internaldec(v)
            lossEstructura = calcularLossEstructura(cl, node)
            lossAtrs = calcularLossAtributo( node, radius )
            nd = createNode(1, radius, cl_prob = lossAtrs , ce = lossEstructura, mse = lossAtrs) 
            
            nodoSiguiente = node.right
            #if not node is None:
            #    if not node.right is None:
            #        nodoSiguiente = node.right
            #    else:
            #        print("aca")
            #        nodoSiguiente = None
            #else:
            #    nodoSiguiente = None
            
            if nodoSiguiente is not None:
                nd.right = decode_node(right, nodoSiguiente, max_nodes, max_depth, level=level+1)
                level=level-1
            return nd

        elif node.childs() == 2 and createNode.count <= max_nodes:
            left, right, radius = bifdec(v)
            lossEstructura = calcularLossEstructura(cl, node)
            lossAtrs = calcularLossAtributo( node, radius )
            nd = createNode(1, radius, cl_prob = lossAtrs, ce = lossEstructura, mse = lossAtrs)
            
            nodoSiguienteRight = node.right
            nodoSiguienteLeft = node.left
            if nodoSiguienteRight is not None:
                nd.right = decode_node(right, nodoSiguienteRight, max_nodes, max_depth, level=level+1)
                level=level-1
            if nodoSiguienteLeft is not None:
                nd.left  = decode_node(left, nodoSiguienteLeft, max_nodes, max_depth, level=level+1)
                level=level-1
            return nd
            
    createNode.count = 0
    dec = decode_node (v, root, max_nodes, max_depth, level=0)
    return dec


In [219]:
def traversefeatures(root, features):
       
    if root is not None:
        traversefeatures(root.left, features)
        features.append(root.radius)
        traversefeatures(root.right, features)
        return features

def numerar_nodos(root, count):
    if root is not None:
        numerar_nodos(root.left, count)
        root.data = len(count)
        count.append(1)
        numerar_nodos(root.right, count)
        return 


def norm(root, minx, miny, minz, minr, maxx, maxy, maxz, maxr):
    
    if root is not None:
        mx = minx.clone().detach()
        my = miny.clone().detach()
        mz = minz.clone().detach()
        mr = minr.clone().detach()
        Mx = maxx.clone().detach()
        My = maxy.clone().detach()
        Mz = maxz.clone().detach()
        Mr = maxr.clone().detach()

        root.radius[0] = (root.radius[0] - minx)/(maxx - minx)
        root.radius[1] = (root.radius[1] - miny)/(maxy - miny)
        root.radius[2] = (root.radius[2] - minz)/(maxz - minz)
        root.radius[3] = (root.radius[3] - minr)/(maxr - minr)
        
        norm(root.left, mx, my, mz, mr, Mx, My, Mz, Mr)
        norm(root.right, mx, my, mz, mr, Mx, My, Mz, Mr)
        return 

def normalize_features(root):
    features = []
    features = traversefeatures(root, features)
    
    x = [tensor[0] for tensor in features]
    y = [tensor[1] for tensor in features]
    z = [tensor[2] for tensor in features]
    r = [tensor[3] for tensor in features]
 
    norm(root, min(x), min(y), min(z), min(r), max(x), max(y), max(z), max(r))

    return 

In [224]:
t_list = ['ArteryObjAN1-2.dat']
class tDataset(Dataset):
    def __init__(self, transform=None):
        self.names = t_list
        self.transform = transform

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        file = t_list[idx]
        string = read_tree("./trees/"+file)
        return string

dataset = tDataset()
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, drop_last=True)


In [243]:
epochs = 1200
learning_rate = 1e-4

params = list(leafenc.parameters()) + list(nonleafenc.parameters()) + list(nodeClassifier.parameters()) + list(featuredec.parameters()) + list(bifdec.parameters())+ list(internaldec.parameters())
opt = torch.optim.Adam(params, lr=learning_rate)

train_loss_avg = []
ce_avg = []
mse_avg = []
lr_list = []
    
for epoch in range(epochs):
    train_loss_avg.append(0)
    ce_avg.append(0)
    mse_avg.append(0)
    lr_list.append(0)
    batches = 0
    for data in data_loader:
            
        d_data = deserialize(data[0])
        li = []
        d_data.traverseInorderChilds(d_data, li)
        normalize_features(d_data)
        enc_fold_nodes = encode_structure_fold(d_data).to(device)
        max_depth = 30
        max_nodes = 200
        decoded = decode_structure_fold(enc_fold_nodes, d_data, max_nodes, max_depth)
           
        l = []
        mse_loss_list = decoded.traverseInorderMSE(decoded, l)
        l = []
        ce_loss_list = decoded.traverseInorderCE(decoded, l)
            
        mse_loss = sum(mse_loss_list) / len(mse_loss_list)
        ce_loss  = sum(ce_loss_list)  / len(ce_loss_list)
        total_loss = (ce_loss + 0.1*mse_loss)

        count = []
        in_n_nodes = len(d_data.count_nodes(d_data, count))
          
        opt.zero_grad()
        total_loss.backward()
        opt.step()
        
        train_loss_avg[-1] += (total_loss.item())
        ce_avg[-1] += (ce_loss.item())
        mse_avg[-1] += (mse_loss.item())
        batches += 1

    train_loss_avg[-1] /= batches
    ce_avg[-1] /= batches
    mse_avg[-1] /= batches
    if epoch % 10 == 0:
        print('Epoch [%d / %d] average reconstruction error: %f mse: %f, ce: %f, lr: %f' % (epoch+1, epochs, train_loss_avg[-1], mse_avg[-1], ce_avg[-1], lr_list[-1]))
            

    

C:\Users\User\AppData\Local\Temp\ipykernel_4160\2448251074.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rad = torch.tensor(input.radius)
C:\Users\User\AppData\Local\Temp\ipykernel_4160\2448251074.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  radius = self.l1(torch.tensor(input.radius).reshape(1,4).to(device))


Epoch [1 / 1200] average reconstruction error: 0.012574 mse: 0.022759, ce: 0.010298, lr: 0.000000
Epoch [11 / 1200] average reconstruction error: 0.015657 mse: 0.023906, ce: 0.013266, lr: 0.000000
Epoch [21 / 1200] average reconstruction error: 0.012953 mse: 0.022738, ce: 0.010679, lr: 0.000000
Epoch [31 / 1200] average reconstruction error: 0.012020 mse: 0.022664, ce: 0.009754, lr: 0.000000
Epoch [41 / 1200] average reconstruction error: 0.011840 mse: 0.022612, ce: 0.009578, lr: 0.000000
Epoch [51 / 1200] average reconstruction error: 0.011576 mse: 0.022460, ce: 0.009330, lr: 0.000000
Epoch [61 / 1200] average reconstruction error: 0.011292 mse: 0.022258, ce: 0.009066, lr: 0.000000
Epoch [71 / 1200] average reconstruction error: 0.011097 mse: 0.022146, ce: 0.008882, lr: 0.000000
Epoch [81 / 1200] average reconstruction error: 0.010946 mse: 0.022125, ce: 0.008733, lr: 0.000000
Epoch [91 / 1200] average reconstruction error: 0.010743 mse: 0.021887, ce: 0.008554, lr: 0.000000
Epoch [101 

KeyboardInterrupt: 

In [231]:
def decode_testing(v, max):
    
    def decode_node(v):
        cl = nodeClassifier(v)
        _, label = torch.max(cl, 1)
        label = label.data
        
        if label == 0 and createNode.count < max: ##output del classifier
            radio = featuredec(v)
            nd = createNode(1,radio)
            return nd

        elif label == 1 and createNode.count < max:
            right, radius = internaldec(v)
            nd = createNode(1, radius) 
            nd.right = decode_node(right)
            return nd

        elif label == 2 and createNode.count < max:
            left, right, radius = bifdec(v)
            nd = createNode(1, radius)
            nd.right = decode_node(right)
            nd.left  = decode_node(left)
            return nd
    
    createNode.count = 0
    dec = decode_node (v)
    return dec

In [232]:
input = deserialize(iter(data_loader).next()[0])
normalize_features(input)
input.traverseInorder(input)
encoded = encode_structure_fold(input).to(device)
print("encoded", enc_fold_nodes)

10 tensor([0.4490, 0.3835, 0.2926, 0.7922], device='cuda:0')
11 tensor([0.5112, 0.4319, 0.3092, 0.7855], device='cuda:0')
12 tensor([0.5662, 0.4702, 0.3241, 0.7940], device='cuda:0')
13 tensor([0.6067, 0.5095, 0.3300, 0.8085], device='cuda:0')
14 tensor([0.6504, 0.5486, 0.3556, 0.8109], device='cuda:0')
15 tensor([0.7006, 0.5836, 0.3625, 0.8291], device='cuda:0')
21 tensor([0.7533, 0.9166, 0.4106, 0.4275], device='cuda:0')
22 tensor([0.7374, 0.9443, 0.4777, 0.0000], device='cuda:0')
16 tensor([0.7755, 0.6896, 0.3731, 0.8923], device='cuda:0')
18 tensor([0.8530, 0.7216, 0.3719, 0.8838], device='cuda:0')
17 tensor([1., 1., 0., 1.], device='cuda:0')
9 tensor([0.3951, 0.3444, 0.2843, 0.7940], device='cuda:0')
8 tensor([0.3201, 0.2997, 0.2775, 0.8339], device='cuda:0')
7 tensor([0.2511, 0.2547, 0.2967, 0.8125], device='cuda:0')
6 tensor([0.1999, 0.2023, 0.3313, 0.7864], device='cuda:0')
5 tensor([0.1627, 0.1575, 0.3763, 0.7699], device='cuda:0')
4 tensor([0.1228, 0.1181, 0.4348, 0.7539], de

C:\Users\User\AppData\Local\Temp\ipykernel_4160\2448251074.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rad = torch.tensor(input.radius)
C:\Users\User\AppData\Local\Temp\ipykernel_4160\2448251074.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  radius = self.l1(torch.tensor(input.radius).reshape(1,4).to(device))


In [233]:
decoded = decode_testing(encoded, 150)
count = []
numerar_nodos(decoded, count)

In [234]:
import meshplot as mp

In [235]:
def plotTree( root, dec ):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    print("plot tree", dec)
    p = mp.plot( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading={'point_size':0.5}, return_plot=True)

    for arista in graph.edges:
        p.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'])

In [238]:
decoded.traverseInorder(decoded)
plotTree (decoded, dec = True)

0 tensor([[0.3475, 0.2912, 0.3523, 0.7640]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
1 tensor([[0.2730, 0.2591, 0.3898, 0.7713]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
2 tensor([[0.2996, 0.3010, 0.4240, 0.8280]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
3 tensor([[0.3527, 0.3620, 0.4642, 0.8708]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
4 tensor([[0.4776, 0.4385, 0.4942, 0.8516]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
5 tensor([[-0.1220, -0.1930,  1.1707,  0.9564]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
6 tensor([[0.3957, 0.3447, 0.2846, 0.7939]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
7 tensor([[0.2639, 0.1229, 0.2667, 0.7591]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
8 tensor([[0.2084, 0.1554, 0.3020, 0.6135]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
9 tensor([[0.2381, 0.1957, 0.3387, 0.7027]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
10 tensor([[0.2503, 0.2231, 0.3629, 0.7394]], 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1778181…

In [ ]:
plt.figure(figsize=(20,10))
nx.draw(G, node_size = 150, with_labels = True)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
fig = plt.plot(train_loss_avg) 
#plt.savefig("loss.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
ax.plot(mse_avg, label="MSE")
ax2 = ax.twinx()
ax2.plot(ce_avg, color="red", label="Cross Entropy")
ax.legend(loc=1)
ax.set_ylim(0, max(mse_avg))

ax2.legend(loc=3)
ax2.set_ylim(0, max(ce_avg))
   

In [ ]:

import sys
from turtle import pos

import meshplot as mp
from vec3 import Vec3
    
grafo = G

posiciones = nx.get_node_attributes( grafo, 'posicion')

li = posiciones[0].toList()
v3 = posiciones[0]
print("vec3", v3)
v3.tocpu()
print("vec3", v3)
'''
li = [x.cpu().detach() for x in li]
print(li)
posiciones.update((x, y.toList()) for x, y in posiciones.items())
print(posiciones[0])

posiciones.update((x, [a.cpu().detach() for a in y]) for x, y in posiciones.items())
print(posiciones)
p = mp.plot( np.array([ np.array(posiciones[node]) for node in grafo.nodes]), return_plot=True, shading={'point_size':4})



for arista in grafo.edges:
    #print("ariste", grafo.nodes[arista[0]]['posicion'].toList())
    arista0 = grafo.nodes[arista[0]]['posicion'].toList()
    arista1 = grafo.nodes[arista[1]]['posicion'].toList()
    print("ariste", arista0)
    arista_0 = [x.cpu().detach() for x in arista0]
    print("ariste", arista_0)
    arista_1 = [x.cpu().detach() for x in arista1]
    p.add_lines( arista_0, arista_1)
'''